In [1]:
import ee
import geemap
import geopandas as gpd
from tagee import terrainAnalysis

ee.Authenticate()

ee.Initialize()
ee.ServiceAccountCredentials(email = "ajs-gee-access@earthengineajs.iam.gserviceaccount.com", 
                              key_file="/Users/Anthony/Documents/GoogleCloud/API_Key/earthengineajs-ajs_gee_access.json")

/opt/miniconda3/envs/geospatial-processing/lib/python3.13/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
import os
from google.cloud import storage

# Set up authentication for Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/Anthony/Documents/GoogleCloud/API_Key/earthengineajs-ajs_gee_access.json"

# Test connection
client = storage.Client()
buckets = list(client.list_buckets())
print(f"Available buckets: {[bucket.name for bucket in buckets]}")

Available buckets: ['ajs_gee_data']


In [3]:
storage_client = storage.Client()
bucket_name = "ajs_gee_data"
bucket = storage_client.bucket(bucket_name)
bucket

<Bucket: ajs_gee_data>

In [4]:
wa = (
    ee.FeatureCollection('USGS/WBD/2017/HUC12')
    .filter(ee.Filter.stringContains("states", "WA"))
    .union()
     )

In [5]:
Map = geemap.Map()

Map.addLayer(wa)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
wa_10m_dem = (
    ee.Image("USGS/3DEP/10m")
    .select("elevation")
    .clip(wa)
    .toFloat()
)

In [11]:
wa_10m_terrain = terrainAnalysis(wa_10m_dem).toFloat()

In [12]:
projection = wa_10m_dem.select('elevation').projection().getInfo()
#projection
task = ee.batch.Export.image.toCloudStorage(
    image=wa_10m_terrain, 
    description='WA_10m_Indices',
    bucket=bucket_name,
    fileNamePrefix='GEE_Export',
    crs=projection['crs'],
    crsTransform=projection['transform'],
    region=wa.geometry(),
    maxPixels=5e10,
    scale=10,
    fileFormat="GeoTIFF"
)
task.start()